# NLTK 기본

## Coursera 수업내용

In [2]:
import nltk
from nltk.corpus import wordnet as wn

In [3]:
# synset 객체를 생성한다
deer = wn.synset("deer.n.01")
elk = wn.synset("elk.n.01")

In [4]:
# path similarity 계산
deer.path_similarity(elk)

0.5

In [5]:
# Lin similarity 계산
from nltk.corpus import wordnet_ic

brown_ic = wordnet_ic.ic("ic-brown.dat")
deer.lin_similarity(elk, brown_ic)

0.8623778273893673

## 토큰생성

자연어 문서를 분석하기 위해서는 우선 긴 문자열을 분석을 위한 작은 단위로 나누어야 한다. 이 문자열 단위를 토큰(token)이라고 하고 이렇게 문자열을 토큰으로 나누는 작업을 토큰 생성(tokenizing)이라고 한다. 영문의 경우에는 문장, 단어 등을 토큰으로 사용하거나 정규 표현식을 쓸 수 있다.

문자열을 토큰으로 분리하는 함수를 토큰 생성 함수(tokenizer)라고 한다. 토큰 생성 함수는 문자열을 입력받아 토큰 문자열의 리스트를 출력한다.

* sent_tokenize -> 문장단위로 tokenize
* word_tokenize -> 단어단위로 tokenize

In [1]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd

In [2]:
# 텍스트 파일을 불러왔다. 이걸로 토큰을 생성할 수 있다.
emma_raw = nltk.corpus.gutenberg.raw("austen-emma.txt")
emma_raw[:50]

'[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\n'

In [3]:
# word_tokenize
from nltk.tokenize import word_tokenize
word_tokenize(emma_raw)[:10]

['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER']

In [5]:
# sentence_tokenize
from nltk.tokenize import sent_tokenize
print(sent_tokenize(emma_raw[:1000])[3])

Sixteen years had Miss Taylor been in Mr. Woodhouse's family,
less as a governess than a friend, very fond of both daughters,
but particularly of Emma.


## 품사부착

품사(POS, part-of-speech) / NLTK에서는 펜 트리뱅크 태그세트(Penn Treebank Tagset)라는 것을 이용한다. 다음은 펜 트리뱅크 태그세트에서 사용하는 품사의 예

NNP: 단수 고유명사  /  VB: 동사

VBP: 동사 현재형  /  TO: to 전치사

NN: 명사(단수형 혹은 집합형)  /  DT: 관형사

In [5]:
from nltk.tag import pos_tag

sentence = "Emma refused to permit us to obtain the refuse permit"
tagged_list = pos_tag(word_tokenize(sentence))
tagged_list

[('Emma', 'NNP'),
 ('refused', 'VBD'),
 ('to', 'TO'),
 ('permit', 'VB'),
 ('us', 'PRP'),
 ('to', 'TO'),
 ('obtain', 'VB'),
 ('the', 'DT'),
 ('refuse', 'NN'),
 ('permit', 'NN')]

Scikit-Learn 등에서 자연어 분석을 할 때는 같은 토큰이라도 품사가 다르면 다른 토큰으로 처리해야 하는 경우가 많은데 이 때는 원래의 토큰과 품사를 붙여서 새로운 토큰 이름을 만들어 사용하면 철자가 같고 품사가 다른 단어를 구분할 수 있다.

In [14]:
def tokenizer(doc):
    return ["/".join(p) for p in tagged_list]

tokenizer(sentence)

['Emma/NNP',
 'refused/VBD',
 'to/TO',
 'permit/VB',
 'us/PRP',
 'to/TO',
 'obtain/VB',
 'the/DT',
 'refuse/NN',
 'permit/NN']

## 단어의 벡터화

* CountVectorizer
* Tfidf

파라미터에는 min_df, ngram_range를 사용할 수 있다 단어의 수를 조절하고, 단어의 어순을 고려할 수 있다

In [8]:
import pandas as pd
import numpy as np

# 데이터불러와서 일부만 사용하기
df = pd.read_csv('Data/Amazon_Unlocked_Mobile.csv')
df = df.sample(frac=0.1, random_state=10)

# Drop missing values
df.dropna(inplace=True)

# Remove any 'neutral' ratings equal to 3
df = df[df['Rating'] != 3]

# Encode 4s and 5s as 1 (rated positively)
# Encode 1s and 2s as 0 (rated poorly)
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df.head(5)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
34377,Apple iPhone 5c 8GB (Pink) - Verizon Wireless,Apple,194.99,1,"The phone needed a SIM card, would have been n...",1.0,0
248521,Motorola Droid RAZR MAXX XT912 M Verizon Smart...,Motorola,174.99,5,I was 3 months away from my upgrade and my Str...,3.0,1
167661,CNPGD [U.S. Office Extended Warranty] Smartwat...,CNPGD,49.99,1,an experience i want to forget,0.0,0
73287,Apple iPhone 7 Unlocked Phone 256 GB - US Vers...,Apple,922.00,5,GREAT PHONE WORK ACCORDING MY EXPECTATIONS.,1.0,1
277158,Nokia N8 Unlocked GSM Touch Screen Phone Featu...,Nokia,95.00,5,I fell in love with this phone because it did ...,0.0,1


In [9]:
# Train-set과 Test-set을 분리한다
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['Reviews'], df['Positively Rated'], 
                                                    random_state=0)

CountVectorizer

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# Fit the CountVectorizer to the training data
vect = CountVectorizer().fit(X_train)

In [12]:
vect.get_feature_names_out()[::2000], len(vect.get_feature_names_out())

(array(['00', 'arroja', 'comapañias', 'dvds', 'golden', 'lands', 'oil',
        'razonable', 'smallsliver', 'tweak'], dtype=object),
 19601)

In [14]:
# 모델 학습에 사용하기 위해선 raw_data를 벡터로 변환해야한다
X_train_vectorized = vect.transform(X_train)

# 이후 모형을 학습시킬 수 있다
from sklearn.linear_model import LogisticRegression
# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorized, y_train)

LogisticRegression(max_iter=1000)

Tfidf

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# min_df는 최소 5번 이상 등장한 단어를 가져온다
vect = TfidfVectorizer(min_df=5).fit(X_train)

In [17]:
# tfidf는 count보다 피쳐의 수가 줄어든 것을 확인할 수 있다
vect.get_feature_names_out()[::2000], len(vect.get_feature_names_out())

(array(['00', 'friends', 'responses'], dtype=object), 5442)

In [18]:
# 똑같이 모형을 학습시켜줄 수 있다
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression()